In [16]:
import pandas as pd
import numpy as np
import scipy
import ot
import matplotlib.pyplot as plt

def norm_and_center_coordinates(X): 
    """
    param: X - numpy array
    
    return: 
    """
    return (X-X.mean(axis=0))/min(scipy.spatial.distance.pdist(X))

def match_spots_using_spatial_heuristic(X,Y,use_ot=True):
    """
    param: X - numpy array
    param: Y - numpy array
    
    return: pi- mapping of spots using spatial heuristic
    """
    n1,n2=len(X),len(Y)
    X,Y = norm_and_center_coordinates(X),norm_and_center_coordinates(Y)
    dist = scipy.spatial.distance_matrix(X,Y)
    if use_ot:
        pi = ot.emd(np.ones(n1)/n1, np.ones(n2)/n2, dist)
    else:
        row_ind, col_ind = scipy.sparse.csgraph.min_weight_full_bipartite_matching(scipy.sparse.csr_matrix(dist))
        pi = np.zeros((n1,n2))
        pi[row_ind, col_ind] = 1/max(n1,n2)
        if n1<n2: pi[:, [(j not in col_ind) for j in range(n2)]] = 1/(n1*n2)
        elif n2<n1: pi[[(i not in row_ind) for i in range(n1)], :] = 1/(n1*n2)
    return pi

def mapping_accuracy(labels1,labels2,pi):
    mapping_dict = {'Layer1':1, 'Layer2':2, 'Layer3':3, 'Layer4':4, 'Layer5':5, 'Layer6':6, 'WM':7}
    return np.sum(pi*(scipy.spatial.distance_matrix(np.matrix(labels1.map(mapping_dict) ).T,np.matrix(labels2.map(mapping_dict)).T)==0))

def get_coords(df, which_coords = ("x", "y"), num_slices=4):
    return [df.loc[df["sample"] == i,[which_coords[0], which_coords[1]]].to_numpy() for i in range(1,num_slices+1)]


def add_clusters(df_coords, df_colinfo, which_sample=(1, 151673)):
    d_slice_coords = dict(tuple(df_coords.groupby('sample'))) 
    
    # create a dictionary where every key is unique name in sample_name colum; every value is a df 
    d_colinfo = dict(tuple(df_colinfo.groupby('sample_name'))) 
    df_final_clusts = d_colinfo[which_sample[1]].dropna(subset=["layer_guess_reordered"])[["barcode", "layer_guess_reordered"]]

    return d_slice_coords[which_sample[0]].merge(df_final_clusts, how="right", on="barcode").dropna()

def compute_pair_accs(df_coords, col_data, p, which_coords=["x", "y"]):
    accs, pis, coords, ids = [], [], [], []
    for (i, (first, second)) in enumerate(zip(p, p[1:])): # Iterate over consecutive pairs
        print(first, second)
        df_first = add_clusters(df_coords, col_data, first)
        df_second = add_clusters(df_coords, col_data, second)
    
        coords_1, coords_2 = df_first[which_coords].to_numpy(), df_second[which_coords].to_numpy()
        pi = match_spots_using_spatial_heuristic(coords_1, coords_2)
        acc = mapping_accuracy(df_first["layer_guess_reordered"], df_second["layer_guess_reordered"], pi)
        accs.append(acc)
        pis.append(pi)
        coords.append(coords_1)
        ids.append(df_first["barcode"].to_list())
        if i == 2:
            coords.append(coords_2)
            ids.append(df_second["barcode"].to_list())
    return accs, pis, coords, ids

In [17]:
base_path = "../data/DLPFC/saved_results/"
res_df = pd.read_csv(base_path + "PASTE_pairwise_accuracy_results_new.csv") 
d_res = dict(tuple(res_df.groupby('kind')))

In [18]:
# Output of STutil labels each slice by a number so for each patient have to label each slice also by 1, 2, 3, 4...
pat_1 = [(i, int(f"15150{i+6}")) for i in range(1,4)] + [(4, int("151510"))]
pat_2 = [(1, int("151669"))] + [(i+1, int(f"15167{i-1}")) for i in range(1,4)]
pat_3 =  [(i, int(f"15167{i+2}")) for i in range(1,4+1)]

col_data = pd.read_csv(base_path + "colDataAll.csv")

In [19]:
datas, pis, cs, bar_ids = [], [], [], []
for (i, pat) in enumerate([pat_1, pat_2, pat_3]):
    df_coords = pd.read_csv(base_path + f"pat-{i+1}-all-slices-STutil-coords-default.csv")
    accs_unalign_xy, _, _, _ = compute_pair_accs(df_coords, col_data, pat)
    print(f"unaligned accs: {accs_unalign_xy}")
    
    accs_align_xy, pis_stutil, coords, barcodes = compute_pair_accs(df_coords, col_data, pat, ["align_x", "align_y"])
    pis.append(pis_stutil)
    print(f"aligned accs: {accs_align_xy}")
    
    data = [accs_unalign_xy, accs_align_xy]
    datas.append(data)
    cs.append(coords)
    bar_ids.append(barcodes)

(1, 151507) (2, 151508)
(2, 151508) (3, 151509)
(3, 151509) (4, 151510)
unaligned accs: [0.8144054891032109, 0.2119327173598592, 0.8690796632495298]
(1, 151507) (2, 151508)
(2, 151508) (3, 151509)
(3, 151509) (4, 151510)
aligned accs: [0.6130960830460331, 0.24303344719555708, 0.8637965061365807]
(1, 151669) (2, 151670)
(2, 151670) (3, 151671)
(3, 151671) (4, 151672)
unaligned accs: [0.902531010850797, 0.5924983793842575, 0.8633977110373154]
(1, 151669) (2, 151670)
(2, 151670) (3, 151671)
(3, 151671) (4, 151672)


/Users/alexanderstrzalkowski/.pyenv/versions/3.8.3/lib/python3.8/site-packages/ot/lp/__init__.py:276: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)


aligned accs: [0.9095759461135535, 0.5782866101374987, 0.4643788602240708]
(1, 151673) (2, 151674)
(2, 151674) (3, 151675)
(3, 151675) (4, 151676)
unaligned accs: [0.8553711588120867, 0.8290505392130079, 0.8406006042037372]
(1, 151673) (2, 151674)
(2, 151674) (3, 151675)
(3, 151675) (4, 151676)
aligned accs: [0.8499714878540596, 0.3054962001664792, 0.14454407890316787]


In [20]:
samples, slice_pairs = ["I", "II", "III"], ["AB", "BC", "CD"]
stutils_res = [[samples[i], slice_pairs[j], "STutility-default", 0.0, datas[i][1][j], np.nan, 0.0, np.nan] for i in range(3) for j in range(3)]
ls =list(range(153,162))  
for (i, l) in enumerate(ls):
    stutils_res[i].insert(0,l)
stutils_df = pd.DataFrame(stutils_res, columns=res_df.columns)
final_df = res_df.append(stutils_df)#.sort_values("Sample")
final_df.to_csv(base_path + "PASTE_pairwise_accuracy_results_new_2", encoding='utf-8', index=False)

In [21]:
pats = [pat_1, pat_2, pat_3]

In [22]:
ids = []
for i in range(3):
    for j in range(4):
        slice_id = pats[i][j][1]
        ids.append(slice_id)

In [23]:
pairs = [f"{first}-{second}" for first, second in zip(ids, ids[1:])]
del pairs[3]
del pairs[6]
print(pairs)

['151507-151508', '151508-151509', '151509-151510', '151669-151670', '151670-151671', '151671-151672', '151673-151674', '151674-151675', '151675-151676']


In [24]:
ss = [s for pi in pis for s in pi]

In [25]:
for (p_id, pi) in zip(pairs, ss):
    print(f"{p_id}-stutil-alignment.csv")
    print(pi.shape)
    #np.savetxt(base_path + f"{p_id}-stutil-alignment.csv", pi, delimiter=',')

151507-151508-stutil-alignment.csv
(4221, 4381)
151508-151509-stutil-alignment.csv
(4381, 4788)
151509-151510-stutil-alignment.csv
(4788, 4595)
151669-151670-stutil-alignment.csv
(3636, 3484)
151670-151671-stutil-alignment.csv
(3484, 4093)
151671-151672-stutil-alignment.csv
(4093, 3888)
151673-151674-stutil-alignment.csv
(3611, 3635)
151674-151675-stutil-alignment.csv
(3635, 3566)
151675-151676-stutil-alignment.csv
(3566, 3431)


In [26]:
for i in range(1,4):
    for j in range(1,5):
        df_tmp = pd.DataFrame(cs[i-1][j-1])
        df_tmp.columns = ["align_x", "align_y"]
        barcode = bar_ids[i-1][j-1]
        df_tmp["barcode"] = barcode
        print(df_tmp.shape)
        slice_id = pats[i-1][j-1][1]
        fn = base_path + f"{slice_id}-STUtil-coords-default.csv"
        #df_tmp.to_csv(fn, index=False)

(4221, 3)
(4381, 3)
(4788, 3)
(4595, 3)
(3636, 3)
(3484, 3)
(4093, 3)
(3888, 3)
(3611, 3)
(3635, 3)
(3566, 3)
(3431, 3)
